In [336]:
import os
import pandas as pd
import math
import matplotlib.pyplot as plt
import ast
import re
from difflib import SequenceMatcher

import torch
from torch import nn

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MultiLabelBinarizer
from sklearn import model_selection


from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import hstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense, LSTM
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Concatenate
from keras.models import Sequential

from tensorflow import keras
from keras import layers

from tensorflow_docs.vis import embed
import tensorflow as tf
import numpy as np
import imageio

In [337]:
dir = os.getcwd()
df = pd.read_csv(dir + r'\data\df_preprocessed.csv')

In [338]:
df = df[:1000]
df = df[['ingredients','label']]

# settings

In [339]:
# percent_testdata = 0.2

In [340]:
# torch.manual_seed(111)

In [341]:
# train_data_length_ex = 1024
# train_data_ex = torch.zeros((train_data_length_ex, 2))
# train_data_ex[:, 0] = 2 * math.pi * torch.rand(train_data_length_ex)
# train_data_ex[:, 1] = torch.sin(train_data_ex[:, 0])
# train_labels_ex = torch.zeros(train_data_length_ex)
# train_set_ex = [
#     (train_data_ex[i], train_labels_ex[i]) for i in range(train_data_length_ex)
# ]

In [342]:
# train_data_ex

# preprocessing

In [343]:
# transform column "ingredients" to list 
df = df.assign(ingredients = [ast.literal_eval(x) for x in df.ingredients])

In [344]:
# remove all words in ingredients which include numbers
df = df.assign(ingredients = [[(''.join([x + ' ' for x in x_sent.split() if not bool(re.search(r'\d', x))]).strip()) for x_sent in x_list] for x_list in df.ingredients])

## one hot encoding

In [345]:
# ordEnc = OrdinalEncoder()
# ohEnc = OneHotEncoder()
mlb = MultiLabelBinarizer(sparse_output=True)

In [346]:
df = df.join(
    pd.DataFrame.sparse.from_spmatrix(
        mlb.fit_transform(df.pop('ingredients')),
        index = df.index, 
        columns = mlb.classes_
    )
)

## join columns with similar names

In [347]:
df

,label,,accent seasoning,acorn squash,active dry yeast,adobo sauce,adobo seasoning,agave nectar,alfredo sauce,all-bran cereal,...,yellow cornmeal,yellow mustard,yellow onion,yellow onions,yellow pepper,yellow squash,yogurt,yukon gold potatoes,zest of,zucchini
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [348]:
# if two column names have a similarity of more than 90%, they are merged
ingredients = []
for column in df.columns:
    # print('\n', column)
    if any(SequenceMatcher(None, ing, column).ratio() > 0.9 for ing in ingredients):
        # print('dublette')
        for ing in ingredients:
            if SequenceMatcher(None, ing, column).ratio() > 0.9:
                df.rename({column: ing}, axis=1, inplace = True) 
                # print(column, ing)
    else:
        # print('no dublette')
        ingredients.append(column)
        # print(ingredients)

In [349]:
# merge columns with same name
df.groupby(level = 0, axis = 1).sum()

,,accent seasoning,acorn squash,active dry yeast,adobo sauce,adobo seasoning,agave nectar,alfredo sauce,all-bran cereal,all-purpose flour,...,yellow cake mix,yellow cornmeal,yellow mustard,yellow onion,yellow pepper,yellow squash,yogurt,yukon gold potatoes,zest of,zucchini
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [350]:
# safe column names for caro weil sie es manuell bearbeiten will :D 

file = open(dir + r'\data\an_caro.csv','w')
for item in list(df.columns):
	file.write(item+"\n")
file.close()

## conversion to PyTorch tensor

In [351]:
# torch_tensor = torch.tensor(df['alfredo sauce'])

In [352]:
# torch_tensor

## conversion to numpy arrays

In [353]:
# y = df['label'].to_numpy()

In [354]:
# X = df[list(df.columns)[2:]].to_numpy()

In [355]:
# X.shape[1]

In [356]:
# # train test split
# trainy, testy = model_selection.train_test_split(y, test_size = 0.2, random_state= 42, shuffle = True)
# trainX, testX = model_selection.train_test_split(X, test_size = 0.2, random_state= 42, shuffle = True)

In [357]:
# trainX[0].shape

In [358]:
# # example of loading the fashion_mnist dataset
# from keras.datasets.fashion_mnist import load_data
# # load the recipes into memory
# (trainX, trainy), (testX, testy) = load_data()
# # summarize the shape of the dataset
# print('Train', trainX.shape, trainy.shape)
# print('Test', testX.shape, testy.shape)

# Conditional GAN
<!-- source of following section: https://machinelearningmastery.com/how-to-develop-a-conditional-generative-adversarial-network-from-scratch/ -->
following section based on: https://keras.io/examples/generative/conditional_gan/

## constants and hyperparameters

In [359]:
batch_size = 64
num_channels = 1
num_classes = 3
feature_size = len(df.columns) - 2
latent_dim = 300

## special preprocessing

In [360]:
df_labels = df['label']
df_features = df.drop(df.columns[[0,1]], axis = 1)

In [361]:
# one hot encoding of labels
df_labels = keras.utils.to_categorical(df_labels, 3)

In [362]:
# convert features dataframe to numpy array
df_features = pd.DataFrame(df_features).to_numpy()

In [363]:
df_labels

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [364]:
# dataset = (
#     tf.data.Dataset.from_tensor_slices(
#         (        
#             tf.cast(df_features.values, tf.float32),
#             tf.cast(df_labels.values, tf.int32)
#         )
#     )
# )

In [365]:
feature_tensor = (tf.convert_to_tensor(df_features))
label_tensor = (tf.convert_to_tensor(df_labels))

In [366]:
# dataset = feature_tensor, label_tensor

In [367]:
# dataset_false = tf.data.Dataset.from_tensor_slices((df_features, df_labels))
# dataset_false = dataset_false.shuffle(buffer_size = 1024).batch(batch_size)

In [368]:
# count = 0
# for features_tensor, target_tensor in dataset:
#     if count == 0:
#         print(f'features:{features_tensor} target:{target_tensor}')
#     count += 1

In [369]:
dataset = tf.data.Dataset.from_tensor_slices((df_features, df_labels))
dataset = dataset.shuffle(buffer_size = 1024).batch(batch_size)

## test

In [370]:
# (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
# all_digits = np.concatenate([x_train, x_test])
# all_labels = np.concatenate([y_train, y_test])

# # Scale the pixel values to [0, 1] range, add a channel dimension to
# # the recipes, and one-hot encode the labels.
# all_digits = all_digits.astype("float32") / 255.0
# all_digits = np.reshape(all_digits, (-1, 28, 28, 1))
# all_labels = keras.utils.to_categorical(all_labels, 10)

# # Create tf.data.Dataset.
# test = tf.data.Dataset.from_tensor_slices((all_digits, all_labels))
# test = test.shuffle(buffer_size=1024).batch(batch_size)

In [371]:
# dataset

In [372]:
# type(all_digits)

## calculating numbers

In [373]:
generator_in_channels = latent_dim + num_classes
discriminator_in_channels = num_channels + num_classes
print(generator_in_channels, discriminator_in_channels)

303 4


In [374]:
# Create the discriminator.
discriminator = keras.Sequential(
    [
        keras.layers.InputLayer((df_features.shape[1], 1, discriminator_in_channels)),
        layers.Conv2D(df_features.shape[1]*2, (3,3), strides = (2, 2), padding = 'same'),
        layers.LeakyReLU(alpha = 0.2),
        layers.Conv2D(df_features.shape[1]*4, (3,3), strides = (2, 2), padding = 'same'),
        layers.LeakyReLU(alpha=0.2),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1),
    ], 
    name = 'discriminator'
)

In [375]:
# # Create the generator.
# generator = keras.Sequential(
#     [
#         keras.layers.InputLayer((generator_in_channels,)),
#         # We want to generate 128 + num_classes coefficients to reshape into a
#         # 7x7x(128 + num_classes) map.
        # layers.Dense(7 * 7 * generator_in_channels), # 7*7+138 = 6762
#         layers.LeakyReLU(alpha=0.2),
#         layers.Reshape((7, 7, generator_in_channels)), # 7,7,138
#         layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"), # 7,7,138
#         layers.LeakyReLU(alpha=0.2),
#         layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"), # 7,7,138
#         layers.LeakyReLU(alpha=0.2),
#         layers.Conv2D(1, (7, 7), padding="same", activation="sigmoid"), #
#     ],
#     name="generator",
# )

In [376]:
# Create the generator.
generator = keras.Sequential(
    [
        keras.layers.InputLayer((generator_in_channels,)),
        # We want to generate 128 + num_classes coefficients to reshape into a
        # 7x7x(128 + num_classes) map.
        layers.Dense(7 * generator_in_channels),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((7, generator_in_channels)),
        layers.Conv1DTranspose(128, 10, strides=2, padding="valid"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv1DTranspose(128, 12, strides=2, padding="valid"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv1DTranspose(128, 10, strides=2, padding="valid"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv1DTranspose(128, 10, strides=2, padding="valid"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv1DTranspose(128, 12, strides=2, padding="valid"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv1DTranspose(128, 12, strides=2, padding="valid"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv1DTranspose(128, 12, strides=2, padding="valid"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv1D(1, 11, padding="valid", activation="sigmoid"),
        # layers.LeakyReLU(alpha=0.2),
        # layers.Conv2D(1, (7, 7), padding="same", activation="sigmoid"),
    ],
    name="generator",
)

## Conditional GAN model

In [392]:
class ConditionalGAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")

    @property
    def metrics(self):
        return [self.gen_loss_tracker, self.disc_loss_tracker]

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, data):
        # Unpack the data.
        real_recipes, one_hot_labels = data
        print(real_recipes)
        real_recipes = tf.cast(real_recipes, tf.float32)

        # Add dummy dimensions to the labels so that they can be concatenated with
        # the recipes. This is for the discriminator.
        recipe_one_hot_labels = one_hot_labels[:, :, None]
        recipe_one_hot_labels = tf.repeat(
            recipe_one_hot_labels, repeats=[feature_size]
        )
        recipe_one_hot_labels = tf.reshape(
            recipe_one_hot_labels, (-1, feature_size, num_classes)
        )
        # print(recipe_one_hot_labels)

        # Sample random points in the latent space and concatenate the labels.
        # This is for the generator.
        batch_size = tf.shape(real_recipes)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )
        
        # Decode the noise (guided by labels) to fake recipes.
        generated_recipes = self.generator(random_vector_labels)

        # Combine them with real recipes. Note that we are concatenating the labels
        # with these recipes here.
        fake_recipe_and_labels = tf.concat([generated_recipes, recipe_one_hot_labels], -1)
        print(real_recipes)
        print(recipe_one_hot_labels)
        real_recipe_and_labels = tf.concat([real_recipes, recipe_one_hot_labels], -1)
        combined_recipes = tf.concat(
            [fake_recipe_and_labels, real_recipe_and_labels], axis=0
        )
        

        # Assemble labels discriminating real from fake recipes.
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        
        # Train the discriminator.
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_recipes)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space.
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Assemble labels that say "all real recipes".
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            fake_recipes = self.generator(random_vector_labels)
            fake_recipe_and_labels = tf.concat([fake_recipes, recipe_one_hot_labels], -1)
            predictions = self.discriminator(fake_recipe_and_labels)
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Monitor loss.
        self.gen_loss_tracker.update_state(g_loss)
        self.disc_loss_tracker.update_state(d_loss)
        return {
            "g_loss": self.gen_loss_tracker.result(),
            "d_loss": self.disc_loss_tracker.result(),
        }

In [393]:
cond_gan = ConditionalGAN(
    discriminator=discriminator, generator=generator, latent_dim=latent_dim
)
cond_gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
)

cond_gan.fit(dataset, epochs=15)

Epoch 1/15
Tensor("IteratorGetNext:0", shape=(None, 1980), dtype=int32)


ValueError: in user code:

    File "c:\Users\I539063\Documents\venvs\DHBW_Semester_6\AML_Scherer\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\I539063\Documents\venvs\DHBW_Semester_6\AML_Scherer\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\I539063\Documents\venvs\DHBW_Semester_6\AML_Scherer\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\I539063\AppData\Local\Temp\ipykernel_28096\2059624283.py", line 24, in train_step
        real_recipes = real_recipes[:, :, 1]

    ValueError: Index out of range using input dim 2; input has only 2 dims for '{{node strided_slice}} = StridedSlice[Index=DT_INT32, T=DT_INT32, begin_mask=3, ellipsis_mask=0, end_mask=3, new_axis_mask=0, shrink_axis_mask=4](It

## Training Conditional GAN

In [ ]:
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 1980), dtype=tf.int32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float32, name=None))>

## Interpolating between classes with the trained generator

In [ ]:
# We first extract the trained generator from our Conditional GAN.
trained_gen = cond_gan.generator

# Choose the number of intermediate recipes that would be generated in
# between the interpolation + 2 (start and last recipes).
num_interpolation = 9  # @param {type:"integer"}

# Sample noise for the interpolation.
interpolation_noise = tf.random.normal(shape=(1, latent_dim))
interpolation_noise = tf.repeat(interpolation_noise, repeats=num_interpolation)
interpolation_noise = tf.reshape(interpolation_noise, (num_interpolation, latent_dim))


def interpolate_class(first_number, second_number):
    # Convert the start and end labels to one-hot encoded vectors.
    first_label = keras.utils.to_categorical([first_number], num_classes)
    second_label = keras.utils.to_categorical([second_number], num_classes)
    first_label = tf.cast(first_label, tf.float32)
    second_label = tf.cast(second_label, tf.float32)

    # Calculate the interpolation vector between the two labels.
    percent_second_label = tf.linspace(0, 1, num_interpolation)[:, None]
    percent_second_label = tf.cast(percent_second_label, tf.float32)
    interpolation_labels = (
        first_label * (1 - percent_second_label) + second_label * percent_second_label
    )

    # Combine the noise and the labels and run inference with the generator.
    noise_and_labels = tf.concat([interpolation_noise, interpolation_labels], 1)
    fake = trained_gen.predict(noise_and_labels)
    return fake


start_class = 1  # @param {type:"slider", min:0, max:9, step:1}
end_class = 5  # @param {type:"slider", min:0, max:9, step:1}

fake_recipes = interpolate_class(start_class, end_class)

IndexError: index 5 is out of bounds for axis 1 with size 3

In [ ]:
# fake_recipes *= 255.0
converted_recipes = fake_recipes.astype(np.uint8)
converted_recipes = tf.image.resize(converted_recipes, (96, 96)).numpy().astype(np.uint8)
imageio.mimsave("animation.gif", converted_recipes, fps=1)
embed.embed_file("animation.gif")